In [1]:
from main import run_main
import json

In [2]:
CROSSOVER_METHODS = ["one_point", "double_point", "uniform", "anular"]
SELECTION_METHODS = ["elite", "roulette", "universal", "deterministic_tournament", "probabilistic_tournament", "ranking"] #borramos Boltzman
MUTATION_METHODS = ["gene_mutation","uniform_multigen", "complete_mutation", "limited_multigen"]
GENES = ["items", "class", "height"]
NEW_GENERATION_METHODS = ["use_all", "new_over_actual"]
CHARACTER_TYPES = ["warrior", "archer", "defender", "infiltrator"]

In [3]:
with open('./config.json', 'r') as f:
        config = json.load(f)

In [4]:
totalIterations = (len(CROSSOVER_METHODS) * len(SELECTION_METHODS)**4 * 
                   len(MUTATION_METHODS) * len(GENES) * len(NEW_GENERATION_METHODS) * 
                   len(CHARACTER_TYPES))

currentIteration = 0

df = []

for cross in CROSSOVER_METHODS:
    for sel1 in SELECTION_METHODS:
        for sel2 in SELECTION_METHODS:
            for sel3 in SELECTION_METHODS:
                for sel4 in SELECTION_METHODS:
                    for mut in MUTATION_METHODS:
                        for gen in GENES:
                            for new_gen in NEW_GENERATION_METHODS:
                                for clase in CHARACTER_TYPES:
                                    
                                    config["character_type"] = clase
                                    config["crossing_method"] = cross
                                    config["gene"] = gen
                                    config["mutation_method"] = mut
                                    config["new_generation_method"] = new_gen
                                    config["selection_method_1"] = sel1
                                    config["selection_method_2"] = sel2
                                    config["selection_method_3"] = sel3
                                    config["selection_method_4"] = sel4
                                    try:
                                        df.append(run_main(config))
                                    except OverflowError:
                                        print(config)

                                    currentIteration += 1
                                    percentageCompletion = (currentIteration / totalIterations) * 100
                                    print(f"\rCompletion: {percentageCompletion:.2f}%", end="")

Completion: 1.50%

KeyboardInterrupt: 

In [ ]:
df